# Extração de contas dos programas do FNDE

Importando os pacotes necessários

In [1]:
import urllib.request

In [2]:
import re

In [3]:
import pandas as pd

In [4]:
import ssl

Desabilitando a verificação do contexto

In [5]:
context = ssl._create_unverified_context()

Importando o banco de dados com as UFs e as URLs das entidades-problema

In [6]:
lista_entidades_problema = pd.read_excel('municipais_problema.xlsx')

In [29]:
lista_entidades_problema.iloc[0][0]

'/sigefweb/index.php/extratos/resultado-contas/programa/C7/esfera/municipal/cnpj/05132436000158?ano=2018&mes=0'

Definindo a url do site do FNDE

In [7]:
site_fnde = 'https://www.fnde.gov.br'

Criando as listas onde serão armazenadas as informações das entidades pesquisadas

In [8]:
lista_ufs = []

In [9]:
lista_urls = []

Carregando as listas com as informações de cada entidade. As expressões regulares foram construídas com a ferramenta disponível no site: http://www.regexlab.com/wild2regex e https://pythex.org/

In [10]:
for entidade in lista_entidades_problema.iterrows():
    with urllib.request.urlopen(site_fnde + entidade[1][0], context=context) as response:
       html = response.read().decode('ansi')
    urls = re.findall(r'<tr><td><a href=\"(.*?)\">', html)
    lista_urls = lista_urls + urls
    for url in urls:
        lista_ufs = lista_ufs + [entidade[1][1]]

Verificar se a quantidade de UFs e URLs são iguais

In [11]:
len(lista_ufs)

4224

In [12]:
len(lista_urls)

4224

Criando um dicionário da entidade com as listas de UFs e URLs

In [13]:
entidades_dict = {'ENTIDADE': lista_urls, 'UF': lista_ufs}

Criando um banco de dados com o dicionário criado

In [14]:
entidades_uf = pd.DataFrame.from_dict(entidades_dict)

In [15]:
lista_cnpjs = []

In [16]:
lista_bancos = []

In [17]:
lista_agencias = []

In [18]:
lista_contas = []

In [19]:
lista_programas = []

In [20]:
for entidade in lista_entidades_problema.iterrows():
    cnpjs = re.findall(r'/cnpj/((?:(?!\?ano)(?:.|\n))*)\?ano', entidade[1][0])
    lista_cnpjs = lista_cnpjs + cnpjs
    programas = re.findall(r'/programa/((?:(?!/esfera/)(?:.|\n))*)/esfera/', entidade[1][0])
    lista_programas = lista_programas + programas
    with urllib.request.urlopen(site_fnde + entidade[1][0], context=context) as response:
       html = response.read().decode('ansi')
    bancos = re.findall(r'>(\w{3})</a></td><td>', html)
    lista_bancos = lista_bancos + bancos
    agencias = re.findall(r'>(\w{4})</a></td><td>', html)
    lista_agencias = lista_agencias + agencias
    contas= re.findall(r'/contacorrente/((?:(?!/programa/)(?:.|\n))*)/programa/', html)
    lista_contas = lista_contas + contas
    esferas = re.findall(r'<tr><td><a href=\"(.*?)\">MUNICIPAL</a>', html)

Conferindo o conteúdo das listas das informações

In [21]:
len(lista_ufs)

4224

In [22]:
len(lista_cnpjs)

1691

In [23]:
len(lista_bancos)

4224

In [24]:
len(lista_agencias)

4224

In [25]:
len(lista_contas)

12672

In [26]:
len(lista_programas)

1691

In [ ]:
len(lista_entidades_problema)

In [ ]:
len(lista_ufs_entidades_problema)

Criando um dicionário com as informações pesquisadas

In [ ]:
entidades = {'UF': lista_ufs, 'CNPJ': lista_cnpjs, 'BANCO': lista_bancos, 'AGENCIA': lista_agencias, 'CONTA': lista_contas, 'PROGRAMA': lista_programas}

In [ ]:
entidades_problema = {'UF': lista_ufs_entidades_problema, 'ENTIDADE': lista_entidades_problema}

Criando um novo banco de dados a partir do dicionário entidades

In [ ]:
banco_entidades = pd.DataFrame.from_dict(entidades)

In [ ]:
banco_entidades_problema = pd.DataFrame.from_dict(entidades_problema)

In [ ]:
banco_entidades.head()

In [ ]:
banco_entidades_problema.head()

In [ ]:
banco_entidades = banco_entidades.drop_duplicates()

In [ ]:
banco_entidades_problema = banco_entidades_problema.drop_duplicates()

In [ ]:
banco_entidades.to_excel('municipais.xlsx', index=False)

In [ ]:
banco_entidades_problema.to_excel('municipais_problema.xlsx', index=False)

In [ ]:
contas = banco_entidades.merge(lista_programas, how='left', left_on='PROGRAMA', right_on='Código')

In [ ]:
contas.drop(columns=['PROGRAMA'])

In [ ]:
contas['PROGRAMA']

In [ ]:
entidades_problema